In [ ]:
import cartopy.crs as ccrs
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pypsa

In [ ]:
zonal_regions = gpd.read_file(
    "/Users/kamrantehranchi/Local_Documents/pypsa-usa/workflow/notebooks/CH1/revision_runs/regions_onshore_s300_134a.geojson"
)

In [ ]:
n = pypsa.Network(
    "/Users/kamrantehranchi/Local_Documents/pypsa-usa/workflow/notebooks/CH1/revision_runs/elec_s300_c134a.nc"
)

In [ ]:
def plot_regions(n, gpd_regions, region_type, text_location="centroid", label_size=10):
    # Create a figure and axis with the same projection as before
    fig, ax = plt.subplots(figsize=(15, 10), subplot_kw={"projection": ccrs.PlateCarree()})

    # Get unique reeds_ba values from n.buses and create a color map
    unique_regions = n.buses[region_type].unique()

    # Create a custom colormap with more distinct colors by combining multiple colormaps
    colors1 = plt.cm.tab20(np.linspace(0, 1, 20))
    colors2 = plt.cm.tab20b(np.linspace(0, 1, 20))
    colors3 = plt.cm.tab20c(np.linspace(0, 1, 20))
    colors = np.vstack((colors1, colors2, colors3))
    # Repeat colors if needed to match number of regions
    n_colors_needed = len(unique_regions)
    n_colors_available = len(colors)
    if n_colors_needed > n_colors_available:
        n_repeats = int(np.ceil(n_colors_needed / n_colors_available))
        colors = np.tile(colors, (n_repeats, 1))
    # Take only as many colors as needed
    colors = colors[:n_colors_needed]
    color_dict = dict(zip(unique_regions, colors))

    # Plot each region with its corresponding color based on reeds_ba
    for r_name in unique_regions:
        # Get the buses in this balancing authority
        ba_buses = n.buses[n.buses[region_type] == r_name].index
        # Filter regions that contain these buses
        mask = gpd_regions.name.isin(ba_buses)
        regions = gpd_regions[mask]
        regions.plot(
            ax=ax,
            facecolor=color_dict[r_name],
            aspect="equal",
            transform=ccrs.PlateCarree(),
            linewidth=0,
            edgecolor=None,
            label=r_name,
        )

        # Calculate and plot the centroid label for this BA
        if text_location == "centroid":
            centroid = regions.union_all().centroid
        else:
            centroid = regions.union_all().representative_point()

        ax.text(
            centroid.x,
            centroid.y,
            r_name,
            transform=ccrs.PlateCarree(),
            color="black",
            ha="center",
            va="center",
            fontsize=label_size,
            fontweight="bold",
        )

    # Set the extent of the plot
    ax.set_extent(gpd_regions.total_bounds[[0, 2, 1, 3]])

    # Add title just above the bounds
    bounds = gpd_regions.total_bounds
    title_x = (bounds[0] + bounds[2]) / 2  # Center x-coordinate
    title_y = bounds[3] + (bounds[3] - bounds[1]) * 0.02  # 2% above top bound
    ax.text(
        title_x,
        title_y,
        f"'{region_type}' boundaries",
        transform=ccrs.PlateCarree(),
        ha="center",
        va="bottom",
        fontsize=20,
        fontweight="bold",
    )

    # Remove the box around the plot
    ax.spines["geo"].set_visible(False)

In [ ]:
plot_regions(n, zonal_regions, "interconnect", text_location="centroid", label_size=15)

In [ ]:
region_type = "nerc_reg"
plot_regions(n, zonal_regions, region_type, text_location="centroid", label_size=15)

In [ ]:
region_type = "trans_reg"
plot_regions(n, zonal_regions, region_type, text_location="centroid", label_size=15)

In [ ]:
region_type = "reeds_ba"
plot_regions(n, zonal_regions, region_type, text_location="representative_point", label_size=12)

In [ ]:
plot_regions(n, zonal_regions, "reeds_state", text_location="centroid")

In [ ]:
region_type = "reeds_zone"
plot_regions(n, zonal_regions, region_type, "centroid", 8)